In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
import datetime
import time
import json
from google.colab import files
from tweepy import Cursor
import xlsxwriter
import pandas as pd
import csv
import datetime
from tweepy.streaming import StreamListener
!pip install xlsxwriter  
!pip3 install twint 

     |████████████████████████████████| 153kB 5.9MB/s 


In [8]:
# load/read from file using JSON
secrets = json.loads(open("secrets_Motorola.json","r").read())

api_key = secrets['CONSUMER_KEY']
api_key_secret = secrets['CONSUMER_SECRET']
access_key = secrets['ACCESS_TOKEN_KEY']
access_key_secret = secrets['ACCESS_TOKEN_SECRET']


auth = OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_key, access_key_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

#change per brand
screen_name = "Moto_Support"

In [9]:
def getAudienceProfile():
  
  with open("MotoProfile.json", 'w') as f:
    result = api.get_user("Moto_Support")
    f.write(json.dumps(result._json, indent=3))

getAudienceProfile()

In [ ]:
def getFollowers():
  counter = 1
  with open("followers_Moto_Support.json","w") as fw:
    for pages in Cursor(api.followers, screen_name="Moto_Support", count =1000).pages(500):
      print("Retrieving 1k follower ")
      time.sleep(2)
      for follower in pages:
        fw.write(json.dumps(follower._json) + "\n")


getFollowers()

In [10]:
def dumpTimeline():
  username = "Moto_Support"
  fname="user_timeline_{}.json".format(username)
  
  print("Batch collection of timeline statuses for ", username)

  counter=0
  with open(fname,'w') as f:
    for page in Cursor(api.user_timeline, screen_name=username,count=50).pages(5):
      for status in page:
        f.write(json.dumps(status._json)+"\n")
        counter+=1
  print("Total %d User Timeline Tweets collected" % counter)

dumpTimeline()

Batch collection of timeline statuses for  Moto_Support
Total 250 User Timeline Tweets collected


In [11]:
import json
screen_name = "Moto_Support"

followers_file = 'followers_Moto_Support.json'.format(screen_name)

with open(followers_file) as f:

    reach = []

    for line in f:

      profile = json.loads(line)

      reach.append((profile['screen_name'], profile['followers_count']))

In [12]:
profile_file = 'MotoProfile.json'.format(screen_name)

with open(profile_file) as f:

    profile = json.load(f)

    followers = profile['followers_count']

    tweets = profile['statuses_count']

sum_reach = sum([x[1] for x in reach])

avg_followers = round(sum_reach / followers, 2)

In [13]:
timeline_file = 'user_timeline_Moto_Support.json'.format(screen_name)

with open(timeline_file) as f:

    favorite_count, retweet_count = [], []

    for line in f:

        tweet = json.loads(line)

        favorite_count.append(tweet['favorite_count'])

        retweet_count.append(tweet['retweet_count'])

In [14]:
avg_favorite = round(sum(favorite_count) / tweets, 5)

avg_retweet = round(sum(retweet_count) / tweets, 5)

favorite_per_user = round(sum(favorite_count) / followers, 5)

retweet_per_user = round(sum(retweet_count) / followers, 5)

In [17]:
print("{} followers".format(followers))

print("{} users reached by 1-degree connections".format(sum_reach))

print("Average number of {}'s followers: {}".format(screen_name, avg_followers))

print("Favorited {} times ({} per tweet, {} per user)".format(sum(favorite_count), avg_favorite, favorite_per_user))

print("Retweeted {} times ({} per tweet, {} per user)".format(sum(retweet_count), avg_retweet, retweet_per_user))

95079 followers
126683 users reached by 1-degree connections
Average number of Moto_Support's followers: 1.33
Favorited 31 times (0.00029 per tweet, 0.00033 per user)
Retweeted 1 times (1e-05 per tweet, 1e-05 per user)
